In [1]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv('nadi_pulse_with_text.csv')
df.head()

In [ ]:
df = df[['Patient Age','text_content','Admin Comment']]
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['Admin Comment'] = df['Admin Comment'].fillna(df['Admin Comment'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
def clear_text(text):
    text = text[:-25]
    return text

In [ ]:
df["text_content"] = df["text_content"].apply(clear_text)

In [ ]:
df.head()

In [ ]:
admin_values = df["Admin Comment"].to_list()

In [ ]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
ord_en = OrdinalEncoder()
la = LabelEncoder()

In [ ]:
df['Admin Comment'] = ord_en.fit_transform(df[['Admin Comment']])

In [ ]:
df.head()

In [ ]:
admin_keys = df["Admin Comment"].to_list()

In [ ]:
string_to_remove = "Start nPULSE001"
r_text = "Start"

In [ ]:
def remove_string_from_content(df, column_name, string_to_remove):
    df["nadi_data"] = df[column_name].apply(lambda x: re.sub(string_to_remove, '', x).strip())
    return df

In [ ]:
def remove_start_from_content(df, column_name, r_text):
    df["nadi_data"] = df[column_name].apply(lambda x: re.sub(r_text, '', x).strip())
    return df

In [ ]:
df = remove_string_from_content(df, "text_content", string_to_remove)
df = remove_start_from_content(df, "nadi_data", r_text)

In [ ]:
df = df[["Patient Age","nadi_data","Admin Comment"]]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
def process_nadi_data(column):
    def safe_convert_to_int(row):
        # Convert only non-empty strings to integers
        return [int(value) for value in row.split(',') if value.strip().isdigit()]
    
    # Process the column
    processed_data = column.apply(lambda x: [safe_convert_to_int(row) for row in x.strip().split('\n') if row.strip()])
    return processed_data

# Process the nadi_data column
df['processed_nadi_data'] = process_nadi_data(df['nadi_data'])

# Display the processed data
df[['processed_nadi_data']]

In [ ]:
df.head()

In [ ]:
# Example: Summing all integers in the processed lists for each row
df['sum_of_nadi_data'] = df['processed_nadi_data'].apply(lambda x: sum(sum(sublist) for sublist in x))

# Example: Averaging all integers in the processed lists for each row
df['avg_of_nadi_data'] = df['processed_nadi_data'].apply(
    lambda x: sum(sum(sublist) for sublist in x) / sum(len(sublist) for sublist in x)
)

# # Example: Taking the maximum value from the processed lists for each row
# df['max_of_nadi_data'] = df['processed_nadi_data'].apply(lambda x: max(max(sublist) for sublist in x))

# Display the resulting columns
df[['sum_of_nadi_data', 'avg_of_nadi_data']]

In [ ]:
df.head()

In [ ]:
new_df = df[["Patient Age","sum_of_nadi_data","Admin Comment"]]
new_df.head()

In [ ]:
new_df.info()

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
min_max = MinMaxScaler()
std = StandardScaler()
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

In [ ]:
x = new_df.iloc[:,:-1]
y = new_df["Admin Comment"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y_train

In [ ]:

trf = ColumnTransformer([
    ('trf',StandardScaler(),["sum_of_nadi_data"])
],remainder='passthrough')

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pipe.score(x_test,y_test)*100

In [ ]:
input_data = [[46,24500481]]
pred_df= pd.DataFrame(input_data, columns=['Patient Age' ,'sum_of_nadi_data'])

In [ ]:
pipe.predict(pred_df)